In [41]:
from lxml import etree
import pandas as pd
from tqdm import tqdm
import os



In [42]:
#setup
path = 'output_xml 2023-03-15 AZ'
xml_file = 'DFT_20230316-124132-All Contracts.xml'
excel_file = '2023-03-15 AZ MN Template for External Contracts - Tiered Line Item 15March.xlsx'
sheet_name = 'Tiered Line Item - TEMPLATE'


In [43]:
# Opening excel and xml file
tree = etree.parse(f'{path}/{xml_file}')
root=tree.getroot()
excel = pd.read_excel(f'{path}/{excel_file}', header = 3, sheet_name=f'{sheet_name}')
#load = pd.read_excel(f'{path}/{load_file}')
df_log = pd.read_csv(f'{path}/logs.csv', header=None)

In [24]:
excel["Product ID"]

0         09H4602
1         09H4702
2         09H4802
3         09H5301
4         09H6003
           ...   
126131    09P8620
126132    09P8624
126133    09P9220
126134    09P9320
126135    09P9422
Name: Product ID, Length: 126136, dtype: object

In [45]:
# saving removed products as list
df_log.columns=['t', 'd', 'a', 'c']
df_log[['A', 'B']] = df_log['a'].str.split(': ', 1, expand=True)
removed = df_log[['B']]
removed[['A', 'B']] = df_log['B'].str.split('_', 1, expand=True)
removed['A'] = removed['A'].str.replace("-", "")
removed = removed['A']
removed = list(removed.drop_duplicates(keep='last'))

ValueError: Length mismatch: Expected axis has 6 elements, new values have 4 elements

In [46]:
# removing products from log from excel
for index in tqdm(excel.index):
    if excel['Product ID'][index] in removed:
        excel.drop([index], axis=0, inplace=True)

100%|██████████| 126136/126136 [00:01<00:00, 107651.60it/s]


In [47]:
# count of products
excel_gr = excel.groupby(['Alternate Contract Id','Customer ID', 
'Price Program Name']).size().reset_index(name='counts')


In [48]:
# Getting numbers and list with products
number_contracts = 0
number_lines = 0

list_all_extended = []
Contract = tree.xpath('.//ContractOffer')
for contract in Contract:
    number_contracts += 1
    Program = contract.xpath('.//Program')
    number_programs = 0
    for program in Program:
        number_programs += 1
        Cfd = program.xpath('.//CFDefinitionSpreadSheets')
        number_products = 0
        if program.attrib['SourceStrategyID'] == 'AbbottTieredLineItemPricingStrategy':
            Product = program.xpath('.//ProductLI')
            for product in Product:
                number_products += 1
                number_lines += 1
                Price = product.xpath('.//TieredLIPrices')
                for price in Price:
                    try:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
                        contract.attrib['OrgUnitName'],contract.attrib['DocName'], contract.attrib['Currency'],
                        program.attrib['Name'], program.attrib['BusinessSegment'], program.attrib['PaymentFrequency'],
                        program.attrib['ContractStartDate'],program.attrib['EndDate'],
                        product.attrib['ProductNum'], 
                        product.attrib['StartEffectiveDate'], product.attrib['EndEffectiveDate'],
                        price.attrib['PriceHigh'], price.attrib['TierVolume']])

                    except:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
                        contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
                        program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['PaymentFrequency'],
                        program.attrib['ContractStartDate'],program.attrib['EndDate'],
                        product.attrib['ProductNum'],
                        product.attrib['StartEffectiveDate'], product.attrib['EndEffectiveDate'],
                        price.attrib['AltUomPrice'], price.attrib['TierVolume']])
                        
        elif program.attrib['SourceStrategyID'] == 'AbbottDOLPricingStrategy':
            Discount = program.xpath('.//DiscountOffListLI')
            for discount in Discount:
                number_products += 1
                number_lines += 1
                Price = discount.xpath('.//LIPrices')
                for price in Price:
                    try:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
                        contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
                        program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['PaymentFrequency'],
                        program.attrib['ContractStartDate'],program.attrib['EndDate'],
                        discount.attrib['ProductNum'],
                        discount.attrib['StartEffectiveDate'], discount.attrib['EndEffectiveDate'],
                        price.attrib['PriceHigh'], ''])

                    except:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
                        contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
                        program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['PaymentFrequency'],
                        program.attrib['ContractStartDate'],program.attrib['EndDate'],
                        discount.attrib['ProductNum'],
                        discount.attrib['StartEffectiveDate'], discount.attrib['EndEffectiveDate'],
                        price.attrib['AltUomPrice'], ''])

    
            
  

In [51]:
# cfd
list_all_cfd = []
Contract = tree.xpath('.//ContractOffer')
for contract in Contract:
    Program = contract.xpath('.//Program')
    for program in Program:
        Cfd = program.xpath('.//CFDefinitionSpreadSheets')
                for cfd in Cfd:
                        Cfdspr = program.xpath('.//CFDefSpreadSheetRecords')
                        for cfdspr in Cfdspr:
                                if cfdspr.attrib('Value1')=='ComplianceThreshold':
                                        list_all_cfd.append([contract.attrib['Customer'], contract.attrib['AlternateId'],
                                        program.attrib['Name'], 
                                        cfdspr.attrib['Value2'],'',''])
                                       
                                if cfdspr.attrib('Value1')=="VolumeTimeframe":
                                        list_all_cfd.append([contract.attrib['Customer'], contract.attrib['AlternateId'],
                                        program.attrib['Name'], 
                                        '',cfdspr.attrib['Value2'],''])
                                if cfdspr.attrib('Value1')=='UnitOfPricing':
                                        list_all_cfd.append([contract.attrib['Customer'], contract.attrib['AlternateId'],
                                        program.attrib['Name'], '','',
                                        cfdspr.attrib['Value2']])
                                       
                                       
                                if cfdspr.attrib('Value1')=="VolumeTimeframe":
                                        list_all_cfd.append([contract.attrib['Customer'], contract.attrib['AlternateId'],
                                        program.attrib['Name'], 
                                        '',cfdspr.attrib['Value2']])
                                       


IndentationError: unexpected indent (1309446268.py, line 8)

In [49]:
# dataframes

df_all_extended = pd.DataFrame(list_all_extended)
df_all_extended.columns =['Customer ID','Alternate Contract Id', 'Contract Start Date','Contract End Date',
'Organization', 'Name', 'Currency',
'Price Program Name', 'Business Segment','Tier Timeframe',
'Program Start Date', 'Program End Date',
'Product ID', 'Product Start Date', 'Product End Date', 'Price', 'Units']
excel_extended = excel[['Customer ID','Alternate Contract Id', 'Start Date', 'End Date',
'Organization', 'Name','Currency',
'Price Program Name', 'Business Segment','Tier Timeframe',
'Program Start Date', 'Program End Date',
 'Product ID', 'Product Start Date', 'Product End Date', 'Tier 1 Price','Tier 1 Units']]


In [60]:
df_all_cfd = pd.DataFrame(list_all_cfd)
df_all_cfd.columns =['Customer ID','Alternate Contract Id', 
'Price Program Name', 
'Compliance Threshold', 'Volume Timeframe',' Unit of Pricing']
excel_all_cfd =excel[['Customer ID','Alternate Contract Id', 
'Price Program Name', 
'Compliance Threshold', 'Volume Timeframe', 'Unit of Pricing']]


NameError: name 'list_all_cfd' is not defined

In [50]:
# removing ending for AZ
excel_extended['Product ID'] = excel_extended['Product ID'].str.replace("_4001", "")
excel_extended['Product ID'] = excel_extended['Product ID'].str.replace("_4002", "")
df_all_extended['Product ID'] = df_all_extended['Product ID'].str.replace("_4001", "")
df_all_extended['Product ID'] = df_all_extended['Product ID'].str.replace("_4002", "")

/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/1747344345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Product ID'] = excel_extended['Product ID'].str.replace("_4001", "")
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/1747344345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Product ID'] = excel_extended['Product ID'].str.replace("_4002", "")


In [51]:
print(excel.shape[0])
print(df_all_extended.shape[0])
print(len(removed))

126096
126096
10


In [52]:
# removing price programs and contract ids with -2
for line in tqdm(df_all_extended.index):
    if df_all_extended['Price Program Name'][line][-2:] == '-2':
        df_all_extended['Price Program Name'][line] = df_all_extended['Price Program Name'][line].replace("-2", "")
    if df_all_extended['Alternate Contract Id'][line][-2:] == '-2':
       df_all_extended['Alternate Contract Id'][line] = df_all_extended['Alternate Contract Id'][line].replace("-2", "")




100%|██████████| 126096/126096 [00:00<00:00, 206581.73it/s]


In [56]:
for line in tqdm(df_all_cfd.index):
    if df_all_cfd['Price Program Name'][line][-2:] == '-2':
        df_all_cfd['Price Program Name'][line] = df_all_cfd['Price Program Name'][line].replace("-2", "")
    if df_all_cfd['Alternate Contract Id'][line][-2:] == '-2':
       df_all_cfd['Alternate Contract Id'][line] = df_all_cfd['Alternate Contract Id'][line].replace("-2", "")


NameError: name 'df_all_cfd' is not defined

In [53]:
# extended
df_all_extended['Price'] = df_all_extended.Price.str.extract(r"(\d+\.\d+)")
df_all_extended['Price'] = df_all_extended['Price'].astype(float)
excel_extended['Tier 1 Price'] = excel_extended['Tier 1 Price'].astype(float)

df_all_extended['Units'] = pd.to_numeric(df_all_extended['Units'], downcast="float")
excel_extended['Tier 1 Units'] = pd.to_numeric(excel_extended['Tier 1 Units'], downcast="float")

/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/4215106603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Tier 1 Price'] = excel_extended['Tier 1 Price'].astype(float)
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/4215106603.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Tier 1 Units'] = pd.to_numeric(excel_extended['Tier 1 Units'], downcast="float")


In [54]:
# count of products
df_gr = df_all_extended.groupby(['Alternate Contract Id','Customer ID', 
'Price Program Name']).size().reset_index(name='counts')

In [55]:
# Merging dataframes- counts
excel_gr['Customer ID'] = excel_gr['Customer ID'].astype(int)
df_gr['Customer ID'] = df_gr['Customer ID'].astype(int)

df_merged = pd.merge(df_gr, excel_gr,  how='outer', left_on=['Customer ID','Alternate Contract Id', 'Price Program Name'], right_on = ['Customer ID','Alternate Contract Id', 'Price Program Name'])
df_merged.rename(columns = {'counts_x':'count from xml', 'counts_y':'count from excel'}, inplace = True)


In [36]:
# checking if the same
df_merged['Are the same?'] = ''
for index in tqdm(df_merged.index):
    if df_merged['count from xml'][index] == df_merged['count from excel'][index]:
        df_merged['Are the same?'][index] = 'yes'
    else:
        df_merged['Are the same?'][index] = 'no'
df_merged.to_excel(f'{path}/result counts.xlsx')

  0%|          | 0/782 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/725705275.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'yes'
100%|██████████| 782/782 [00:00<00:00, 11311.37it/s]


In [56]:
# Merging dataframes
excel_extended['Customer ID'] = excel_extended['Customer ID'].astype(int)
df_all_extended['Customer ID'] = df_all_extended['Customer ID'].astype(int)
df_all_extended['Product ID'] = df_all_extended['Product ID'].str.replace("-", "")
df_merged_extended = pd.merge(df_all_extended, excel_extended,  how='outer', left_on=['Customer ID','Alternate Contract Id', 'Price Program Name', 'Product ID'], right_on = ['Customer ID', 'Alternate Contract Id', 'Price Program Name','Product ID' ])



/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/3066667302.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Customer ID'] = excel_extended['Customer ID'].astype(int)


In [46]:
# Merging dataframes
excel_all_cfd['Customer ID'] = excel_all_cfd['Customer ID'].astype(int)
df_all_cfd['Customer ID'] = df_all_cfd['Customer ID'].astype(int)
df_merged_cfd = pd.merge(df_all_cfd, excel_all_cfd,  how='outer', left_on=['Customer ID','Alternate Contract Id', 'Price Program Name'], right_on = ['Customer ID', 'Alternate Contract Id', 'Price Program Name'])


NameError: name 'excel_all_cfd' is not defined

In [57]:
# data types 
# data types 
df_merged_extended['Product Start Date_x']=pd.to_datetime(df_merged_extended['Product Start Date_x'])
df_merged_extended['Product Start Date_y']=pd.to_datetime(df_merged_extended['Product Start Date_y'])
df_merged_extended['Product End Date_x']=pd.to_datetime(df_merged_extended['Product End Date_x'])
df_merged_extended['Product End Date_y']=pd.to_datetime(df_merged_extended['Product End Date_y'])
df_merged_extended['Program Start Date_x']=pd.to_datetime(df_merged_extended['Program Start Date_x'])
df_merged_extended['Program Start Date_y']=pd.to_datetime(df_merged_extended['Program Start Date_y'])
df_merged_extended['Program End Date_x']=pd.to_datetime(df_merged_extended['Program End Date_x'])
df_merged_extended['Program End Date_y']=pd.to_datetime(df_merged_extended['Program End Date_y'])
df_merged_extended['Contract End Date']=pd.to_datetime(df_merged_extended['Contract End Date'])
df_merged_extended['Start Date']=pd.to_datetime(df_merged_extended['Start Date'])
df_merged_extended['Contract Start Date']=pd.to_datetime(df_merged_extended['Contract Start Date'])
df_merged_extended['End Date']=pd.to_datetime(df_merged_extended['End Date'])
df_merged_extended['End Date']=pd.to_datetime(df_merged_extended['End Date'])
df_merged_extended['Organization_y']= pd.to_numeric(df_merged_extended['Organization_y'])
df_merged_extended['Organization_x']= pd.to_numeric(df_merged_extended['Organization_x'])
df_merged_extended['Units'] = pd.to_numeric(df_merged_extended['Units'])
df_merged_extended['Tier 1 Units'] = pd.to_numeric(df_merged_extended['Tier 1 Units'])

In [58]:
df_merged_extended['Are prices the same?'] = ''
df_merged_extended['Are start dates the same?'] = ''
df_merged_extended['Are end dates the same?'] = ''
df_merged_extended['Is org the same?'] = ''
df_merged_extended['Is name the same?'] = ''
df_merged_extended['Is program start date the same?'] = ''
df_merged_extended['Is program end date the same?'] = ''
df_merged_extended['Is product start date the same?'] = ''
df_merged_extended['Is product end date the same?'] = ''
df_merged_extended['Are units the same?'] = ''
for index in tqdm(df_merged_extended.index):
    if df_merged_extended['Price'][index] == df_merged_extended['Tier 1 Price'][index]:
        df_merged_extended['Are prices the same?'][index] = 'yes'
    else:
        df_merged_extended['Are prices the same?'][index] = 'no'

    if df_merged_extended['Contract Start Date'][index] == df_merged_extended['Start Date'][index]:
        df_merged_extended['Are start dates the same?'] = 'yes'
    else:
        df_merged_extended['Are start dates the same?'] = 'no'

    if df_merged_extended['Contract End Date'][index] == df_merged_extended['End Date'][index]:
        df_merged_extended['Are end dates the same?'][index] = 'yes'
    else:
        df_merged_extended['Are end dates the same?'][index] = 'no'

    if df_merged_extended['Organization_x'][index] == df_merged_extended['Organization_y'][index]:
        df_merged_extended['Is org the same?'][index] = 'yes'
    else:
        df_merged_extended['Is org the same?'][index] = 'no'

    if df_merged_extended['Name_x'][index] == df_merged_extended['Name_y'][index]:
        df_merged_extended['Is name the same?'][index] = 'yes'
    else:
        df_merged_extended['Is name the same?'][index] = 'no'
        
    if df_merged_extended['Product Start Date_x'][index] == df_merged_extended['Product Start Date_y'][index]:
        df_merged_extended['Is product start date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is product start date the same?'][index] = 'no'

    if df_merged_extended['Product End Date_x'][index] == df_merged_extended['Product End Date_y'][index]:
        df_merged_extended['Is product end date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is product end date the same?'][index] = 'no'

    if df_merged_extended['Program Start Date_x'][index] == df_merged_extended['Program Start Date_y'][index]:
            df_merged_extended['Is program start date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is program start date the same?'][index] = 'no'

    
    if df_merged_extended['Program End Date_x'][index] == df_merged_extended['Program End Date_y'][index]:
            df_merged_extended['Is program end date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is program end date the same?'][index] = 'no'

    if df_merged_extended['Units'][index] == df_merged_extended['Tier 1 Units'][index]:
        df_merged_extended['Are units the same?'][index] = 'yes'
    else:
        df_merged_extended['Are units the same?'][index] = 'no'



  0%|          | 0/126344 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/310394543.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are prices the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/310394543.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are end dates the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_2166/310394543.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

In [ ]:
df_merged_extended['Are prices the same?'] = ''
df_merged_extended['Are start dates the same?'] = ''
df_merged_extended['Are end dates the same?'] = ''
'Compliance Threshold', 'Volume Timeframe', 'Unit of Pricing'
for index in tqdm(df_merged_extended.index):
    if df_merged_extended['Price'][index] == df_merged_extended['Tier 1 Price'][index]:
        df_merged_extended['Are prices the same?'][index] = 'yes'
    else:
        df_merged_extended['Are prices the same?'][index] = 'no'
    if df_merged_extended['Contract Start Date'][index] == df_merged_extended['Start Date'][index]:
        df_merged_extended['Are start dates the same?'] = 'yes'
    else:
        df_merged_extended['Are start dates the same?'] = 'no'
    if df_merged_extended['Contract End Date'][index] == df_merged_extended['End Date'][index]:
        df_merged_extended['Are end dates the same?'][index] = 'yes'
    else:
        df_merged_extended['Are end dates the same?'][index] = 'no'
    if df_merged_extended['Organization_x'][index] == df_merged_extended['Organization_y'][index]:
        df_merged_extended['Is org the same?'][index] = 'yes'
    else:
        df_merged_extended['Is org the same?'][index] = 'no'
    if df_merged_extended['Name_x'][index] == df_merged_extended['Name_y'][index]:
        df_merged_extended['Is name the same?'][index] = 'yes'
    else:
        df_merged_extended['Is name the same?'][index] = 'no'
    if df_merged_extended['Product Start Date_x'][index] == df_merged_extended['Product Start Date_y'][index]:
        df_merged_extended['Is program start date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is program start date the same?'][index] = 'no'
    if df_merged_extended['Product End Date_x'][index] == df_merged_extended['Product End Date_y'][index]:
        df_merged_extended['Is program end date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is program end date the same?'][index] = 'no'
    if df_merged_extended['Tier Timeframe_x'][index] == df_merged_extended['Tier Timeframe_y'][index][:2].upper():
        df_merged_extended['Is Tier Timeframe the same?'][index] = 'yes'



In [40]:

df_merged_extended.to_excel(f'{path}/result extended2.xlsx')